In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
%matplotlib notebook
from Reinforcement_Learning import *
from Maze import *
from DrawMaze import *
import cv2
import matplotlib.pyplot as plt

In [2]:
#迷路生成
PATH = 0 #道
WALL = 1 #壁
STR = 'S' #スタート
GOL = 'G' #ゴール
mWidth, mHeight = 20, 20
maze = Maze(mWidth, mHeight)
maze.set_out_wall()
maze.set_inner_wall_botaoshi()
maze.set_start_goal()
maze.print_maze()
mWidth, mHeight = maze.get_width_height()
# 迷路データ maze.maze[x][y]

＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃
＃Ｓ＃　＃　＃　＃　　　＃　　　　　＃　＃
＃　＃　＃　＃　＃　＃＃＃　＃＃＃＃＃　＃
＃　　　　　　　　　　　　　＃　＃　　　＃
＃　＃＃＃　＃　＃＃＃　＃　＃　＃　＃＃＃
＃　　　＃　＃　　　＃　＃　　　＃　　　＃
＃　＃　＃　＃　＃　＃＃＃　＃＃＃　＃＃＃
＃　＃　＃　＃　＃　　　＃　　　　　＃　＃
＃　＃＃＃　＃　＃＃＃＃＃　＃＃＃　＃　＃
＃　　　＃　＃　＃　　　＃　＃　＃　＃　＃
＃　＃＃＃　＃　＃　＃＃＃　＃　＃　＃　＃
＃　＃　＃　＃　　　＃　＃　　　＃　＃　＃
＃　＃　＃＃＃＃＃＃＃　＃＃＃　＃＃＃　＃
＃　　　　　　　　　　　＃　＃　　　＃　＃
＃　＃　＃＃＃　＃＃＃　＃　＃　＃＃＃　＃
＃　＃　　　＃　　　＃　＃　＃　　　＃　＃
＃＃＃　＃＃＃　＃＃＃＃＃　＃＃＃＃＃　＃
＃　　　　　＃　　　　　　　＃　　　　　＃
＃　＃　＃＃＃　＃　＃　＃＃＃　＃　＃＃＃
＃　＃　　　＃　＃　＃　　　　　＃　　Ｇ＃
＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃


In [3]:
# 強化学習
# q値更新
learningRate = 0.1 #学習係数α
discountRate = 0.9 #割引率γ
qUpdate = QLearning(learningRate, discountRate) # Q学習

#行動選択方法
ep = 0.3 #ε
actionSelect = EpGreedy(ep) #ε-greedy 

learning = ReinforcementLearning(mWidth, mHeight, qUpdate, actionSelect)

In [4]:
#壁でないか確認
def is_not_wall(x, y):
    if maze.maze[x][y] == WALL:
        return False
    else:
        return True

In [5]:
# 迷路描画機能
imgW = 800
imgH = 800
draw = DrawMaze(imgW, imgH, maze.maze)

In [6]:
# 学習初期化
str_x, str_y = maze.get_start() #TODO Start位置の取得関数
gol_x, gol_y = maze.get_goal() #TODO Goal位置の取得関数
gol_reward = 30 #成功報酬
sx, sy = str_x, str_y
maxAction = 10000 # 最大行動数
maxResetAction = (mWidth + mHeight) * 30 # リセットになる行動回数
resetAction = 0

In [7]:
for t in range(0, maxAction):
    # 次の行動を決定
    while True:
        nx, ny, action = learning.get_next_state(sx, sy)
        # 今回は次の行動が壁の時はキャンセルする
        if is_not_wall(nx, ny):
            break
    
    # Q値更新
    sq = learning.get_q_val(sx, sy, action)
    na, nq = learning.get_q_max_val(nx, ny)
    reward = gol_reward if (nx == gol_x and ny == gol_y) else 0
    p = [sq, nq, reward]
    learning.update_q_val(sx, sy, action, p)
    
    # 迷路描画
    draw.drawMaze(learning, gol_reward, sx, sy, nx, ny)
    sx, sy = nx, ny
    
    resetAction = resetAction + 1
    # リセット回数に達している、またはゴールならばスタートに戻る
    if (resetAction >= maxResetAction) or (sx == gol_x and sy == gol_y):
        resetAction = 0
        sx, sy = str_x, str_y    

In [8]:
#結果出力(動画)
# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# output file name, encoder, fps, size(fit to image size)
video = cv2.VideoWriter('video.mp4',fourcc, 120.0, (imgW, imgH))
if not video.isOpened():
    print("can't be opened")
    sys.exit()
    
for i in range(0, maxAction):
    # hoge0000.png, hoge0001.png,..., hoge0090.png
    img = cv2.imread('output/images/img_' + str(i)+ '.jpg')

    # can't read image, escape
    if img is None:
        print("can't read")
        break

    # add
    video.write(img)
    print("read image " + str(i))

video.release()
print('written')

read image 0
read image 1
read image 2
read image 3
read image 4
read image 5
read image 6
read image 7
read image 8
read image 9
read image 10
read image 11
read image 12
read image 13
read image 14
read image 15
read image 16
read image 17
read image 18
read image 19
read image 20
read image 21
read image 22
read image 23
read image 24
read image 25
read image 26
read image 27
read image 28
read image 29
read image 30
read image 31
read image 32
read image 33
read image 34
read image 35
read image 36
read image 37
read image 38
read image 39
read image 40
read image 41
read image 42
read image 43
read image 44
read image 45
read image 46
read image 47
read image 48
read image 49
read image 50
read image 51
read image 52
read image 53
read image 54
read image 55
read image 56
read image 57
read image 58
read image 59
read image 60
read image 61
read image 62
read image 63
read image 64
read image 65
read image 66
read image 67
read image 68
read image 69
read image 70
read image 71
re

read image 578
read image 579
read image 580
read image 581
read image 582
read image 583
read image 584
read image 585
read image 586
read image 587
read image 588
read image 589
read image 590
read image 591
read image 592
read image 593
read image 594
read image 595
read image 596
read image 597
read image 598
read image 599
read image 600
read image 601
read image 602
read image 603
read image 604
read image 605
read image 606
read image 607
read image 608
read image 609
read image 610
read image 611
read image 612
read image 613
read image 614
read image 615
read image 616
read image 617
read image 618
read image 619
read image 620
read image 621
read image 622
read image 623
read image 624
read image 625
read image 626
read image 627
read image 628
read image 629
read image 630
read image 631
read image 632
read image 633
read image 634
read image 635
read image 636
read image 637
read image 638
read image 639
read image 640
read image 641
read image 642
read image 643
read image

read image 1133
read image 1134
read image 1135
read image 1136
read image 1137
read image 1138
read image 1139
read image 1140
read image 1141
read image 1142
read image 1143
read image 1144
read image 1145
read image 1146
read image 1147
read image 1148
read image 1149
read image 1150
read image 1151
read image 1152
read image 1153
read image 1154
read image 1155
read image 1156
read image 1157
read image 1158
read image 1159
read image 1160
read image 1161
read image 1162
read image 1163
read image 1164
read image 1165
read image 1166
read image 1167
read image 1168
read image 1169
read image 1170
read image 1171
read image 1172
read image 1173
read image 1174
read image 1175
read image 1176
read image 1177
read image 1178
read image 1179
read image 1180
read image 1181
read image 1182
read image 1183
read image 1184
read image 1185
read image 1186
read image 1187
read image 1188
read image 1189
read image 1190
read image 1191
read image 1192
read image 1193
read image 1194
read ima

read image 1654
read image 1655
read image 1656
read image 1657
read image 1658
read image 1659
read image 1660
read image 1661
read image 1662
read image 1663
read image 1664
read image 1665
read image 1666
read image 1667
read image 1668
read image 1669
read image 1670
read image 1671
read image 1672
read image 1673
read image 1674
read image 1675
read image 1676
read image 1677
read image 1678
read image 1679
read image 1680
read image 1681
read image 1682
read image 1683
read image 1684
read image 1685
read image 1686
read image 1687
read image 1688
read image 1689
read image 1690
read image 1691
read image 1692
read image 1693
read image 1694
read image 1695
read image 1696
read image 1697
read image 1698
read image 1699
read image 1700
read image 1701
read image 1702
read image 1703
read image 1704
read image 1705
read image 1706
read image 1707
read image 1708
read image 1709
read image 1710
read image 1711
read image 1712
read image 1713
read image 1714
read image 1715
read ima

read image 2185
read image 2186
read image 2187
read image 2188
read image 2189
read image 2190
read image 2191
read image 2192
read image 2193
read image 2194
read image 2195
read image 2196
read image 2197
read image 2198
read image 2199
read image 2200
read image 2201
read image 2202
read image 2203
read image 2204
read image 2205
read image 2206
read image 2207
read image 2208
read image 2209
read image 2210
read image 2211
read image 2212
read image 2213
read image 2214
read image 2215
read image 2216
read image 2217
read image 2218
read image 2219
read image 2220
read image 2221
read image 2222
read image 2223
read image 2224
read image 2225
read image 2226
read image 2227
read image 2228
read image 2229
read image 2230
read image 2231
read image 2232
read image 2233
read image 2234
read image 2235
read image 2236
read image 2237
read image 2238
read image 2239
read image 2240
read image 2241
read image 2242
read image 2243
read image 2244
read image 2245
read image 2246
read ima

...

read image 7409
read image 7410
read image 7411
read image 7412
read image 7413
read image 7414
read image 7415
read image 7416
read image 7417
read image 7418
read image 7419
read image 7420
read image 7421
read image 7422
read image 7423
read image 7424
read image 7425
read image 7426
read image 7427
read image 7428
read image 7429
read image 7430
read image 7431
read image 7432
read image 7433
read image 7434
read image 7435
read image 7436
read image 7437
read image 7438
read image 7439
read image 7440
read image 7441
read image 7442
read image 7443
read image 7444
read image 7445
read image 7446
read image 7447
read image 7448
read image 7449
read image 7450
read image 7451
read image 7452
read image 7453
read image 7454
read image 7455
read image 7456
read image 7457
read image 7458
read image 7459
read image 7460
read image 7461
read image 7462
read image 7463
read image 7464
read image 7465
read image 7466
read image 7467
read image 7468
read image 7469
read image 7470
read ima

read image 7938
read image 7939
read image 7940
read image 7941
read image 7942
read image 7943
read image 7944
read image 7945
read image 7946
read image 7947
read image 7948
read image 7949
read image 7950
read image 7951
read image 7952
read image 7953
read image 7954
read image 7955
read image 7956
read image 7957
read image 7958
read image 7959
read image 7960
read image 7961
read image 7962
read image 7963
read image 7964
read image 7965
read image 7966
read image 7967
read image 7968
read image 7969
read image 7970
read image 7971
read image 7972
read image 7973
read image 7974
read image 7975
read image 7976
read image 7977
read image 7978
read image 7979
read image 7980
read image 7981
read image 7982
read image 7983
read image 7984
read image 7985
read image 7986
read image 7987
read image 7988
read image 7989
read image 7990
read image 7991
read image 7992
read image 7993
read image 7994
read image 7995
read image 7996
read image 7997
read image 7998
read image 7999
read ima

read image 8468
read image 8469
read image 8470
read image 8471
read image 8472
read image 8473
read image 8474
read image 8475
read image 8476
read image 8477
read image 8478
read image 8479
read image 8480
read image 8481
read image 8482
read image 8483
read image 8484
read image 8485
read image 8486
read image 8487
read image 8488
read image 8489
read image 8490
read image 8491
read image 8492
read image 8493
read image 8494
read image 8495
read image 8496
read image 8497
read image 8498
read image 8499
read image 8500
read image 8501
read image 8502
read image 8503
read image 8504
read image 8505
read image 8506
read image 8507
read image 8508
read image 8509
read image 8510
read image 8511
read image 8512
read image 8513
read image 8514
read image 8515
read image 8516
read image 8517
read image 8518
read image 8519
read image 8520
read image 8521
read image 8522
read image 8523
read image 8524
read image 8525
read image 8526
read image 8527
read image 8528
read image 8529
read ima

read image 8996
read image 8997
read image 8998
read image 8999
read image 9000
read image 9001
read image 9002
read image 9003
read image 9004
read image 9005
read image 9006
read image 9007
read image 9008
read image 9009
read image 9010
read image 9011
read image 9012
read image 9013
read image 9014
read image 9015
read image 9016
read image 9017
read image 9018
read image 9019
read image 9020
read image 9021
read image 9022
read image 9023
read image 9024
read image 9025
read image 9026
read image 9027
read image 9028
read image 9029
read image 9030
read image 9031
read image 9032
read image 9033
read image 9034
read image 9035
read image 9036
read image 9037
read image 9038
read image 9039
read image 9040
read image 9041
read image 9042
read image 9043
read image 9044
read image 9045
read image 9046
read image 9047
read image 9048
read image 9049
read image 9050
read image 9051
read image 9052
read image 9053
read image 9054
read image 9055
read image 9056
read image 9057
read ima

read image 9524
read image 9525
read image 9526
read image 9527
read image 9528
read image 9529
read image 9530
read image 9531
read image 9532
read image 9533
read image 9534
read image 9535
read image 9536
read image 9537
read image 9538
read image 9539
read image 9540
read image 9541
read image 9542
read image 9543
read image 9544
read image 9545
read image 9546
read image 9547
read image 9548
read image 9549
read image 9550
read image 9551
read image 9552
read image 9553
read image 9554
read image 9555
read image 9556
read image 9557
read image 9558
read image 9559
read image 9560
read image 9561
read image 9562
read image 9563
read image 9564
read image 9565
read image 9566
read image 9567
read image 9568
read image 9569
read image 9570
read image 9571
read image 9572
read image 9573
read image 9574
read image 9575
read image 9576
read image 9577
read image 9578
read image 9579
read image 9580
read image 9581
read image 9582
read image 9583
read image 9584
read image 9585
read ima